# AutoBuild 自動建構
AutoGen 提供由 LLM、工具或人員支援的可對話代理，可用於透過自動聊天集體執行任務。 該框架允許透過多代理對話使用工具和人類參與。
請在[此處](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat)尋找有關此功能的文件。

在本筆記本中，我們引入了一個新類別“AgentBuilder”，以幫助使用者建立由多代理系統支援的自動任務解決流程。 具體來說，在`build()`中，我們提示LLM建立多個參與者代理程式並初始化群組聊天，並指定該任務是否需要編程來解決。 AgentBuilder 也透過 [vLLM](https://docs.vllm.ai/en/latest/index.html) 和 [Fastchat](https://github.com/lm-sys/FastChat) 支援開源 LLM。 檢查支援的模型清單[此處](https://docs.vllm.ai/en/latest/models/supported_models.html)。

## Requirement 要求

AutoBuild 需要最新版本的 AutoGen。
您可以透過以下命令安裝 AutoGen：

In [ ]:
%pip install -U --quiet pyautogen

In [2]:
import os
import autogen
from autogen import AssistantAgent, Agent, UserProxyAgent, ConversableAgent

## Step 1: prepare configuration 第 1 步：準備配置
為助理代理準備一個“config_path”，以限制您要在此任務中使用的LLM的選擇。 此配置可以是json檔案的路徑或環境變數的名稱。 初始化 LLM 的特定配置（如種子、溫度等）也需要「default_llm_config」...

In [3]:
config_path = 'OAI_CONFIG_LIST'  # modify path
default_llm_config = {
    'temperature': 0
}
"""
20240219，此處列表更新為5類
gpt35、gpt4、gpt4v、
gemini、gemini_vision(gemini 還不大行，會報錯，暫不支援)
cohere 可以調用，但免費的很快超過額度
groq   可以調用，但有點怪怪的。
也加入本地大模型，但相容性似乎都有問題，第一輪對話還可以，第一輪之後都不行。
- ollama：要使用 litellm
- LM Studio 
- Jan (2024/03/02 開啟GPU加速會報錯)
"""
config_list_gpt35 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-3.5-turbo", "gpt-3.5-turbo-1106", "gpt-3.5-turbo-0613", "gpt-3.5-turbo-16k", "gpt-3.5-turbo-16k-0613"],
    },
)
config_list_ollama = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["ollama"],
    },
)
config_list_LM_Satudio = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["LM Studio"],
    },
)
config_list_jan = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["OpenHermes Neural 7B Q4"],
    },
)
config_list_groq_llama2 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["llama2-70b-4096"],
    },
)
config_list_groq_mixtral = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["mixtral-8x7b-32768"],
    },
)
config_list_cohere = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["command-nightly"],
    },
)
config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0613", "gpt-4-0314", "gpt-4-1106-preview"],
    },
)
config_list_gpt4v = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4-vision-preview", "dalle"],
    },
)
config_list_gemini = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gemini-pro"],
    },
)
config_list_gemini_vision = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gemini-pro-vision"],
    },
)


In [6]:
# 這裡設定要用那一個大模型
config_list = config_list_groq_mixtral

## Step 2: create a AgentBuilder 第二步：建立AgentBuilder
使用指定的“config_path”建立“AgentBuilder”。 AgentBuilder預設使用GPT-4來完成整個過程，您也可以根據需要將“builder_model”更改為其他OpenAI模型。 您也可以指定 OpenAI 或開源 LLM 作為代理主幹，請參閱部落格以了解更多詳細資訊。

In [7]:
from autogen.agentchat.contrib.agent_builder import AgentBuilder

# builder = AgentBuilder(config_path=config_path, builder_model='gpt-3.5-turbo-16k', agent_model='gpt-3.5-turbo-16k')
# builder = AgentBuilder(builder_model='gpt-3.5-turbo-16k', agent_model='gpt-3.5-turbo-16k')
# builder = AgentBuilder(builder_model='gemini-pro', agent_model='gemini-pro')
# builder = AgentBuilder(builder_model='ollama', agent_model='ollama')
# builder = AgentBuilder(builder_model='LM Studio', agent_model='LM Studio')
# builder = AgentBuilder(builder_model='command-nightly', agent_model='command-nightly')  # config_list = config_list_cohere
# builder = AgentBuilder(builder_model='llama2-70b-4096', agent_model='llama2-70b-4096') # config_list_groq_llama2
builder = AgentBuilder(builder_model='mixtral-8x7b-32768', agent_model='mixtral-8x7b-32768') # config_list_groq_mixtral

## Step 3: specify a building task

Specify a building task with a general description. Building task will help build manager (a LLM) decide what agents should be build.

In [8]:
building_task = "Find a paper on arxiv by programming, and analyze its application in some domain. For example, find a recent paper about gpt-4 on arxiv and find its potential applications in software."

## Step 4: build group chat agents
Use `build()` to let build manager (the specified `builder_model`) complete the group chat agents generation. If you think coding is necessary in your task, you can use `coding=True` to add a user proxy (an automatic code interpreter) into the agent list, like: 
```python
builder.build(building_task, default_llm_config, coding=True)
```
If `coding` is not specified, AgentBuilder will determine on its own whether the user proxy should be added or not according to the task.

In [9]:
default_llm_config={"config_list": config_list, "seed": 42}
agent_list, agent_configs = builder.build(building_task, default_llm_config)
# agent_list, agent_configs = builder.build(building_task, llm_config={"config_list": config_list_gemini, "seed": 42})

RuntimeError: Fail to initialize build manager: mixtral-8x7b-32768 does not exist in OAI_CONFIG_LIST. If you want to change this model, please specify the "builder_model" in the constructor.

## Step 5: execute task
Let agents generated in `build()` to complete the task collaboratively in a group chat.

In [18]:
import autogen

def start_task(execution_task: str, agent_list: list, llm_config: dict):
    config_list = autogen.config_list_from_json(config_path, filter_dict={"model": ["gpt-3.5-turbo-1106"]})
    
    group_chat = autogen.GroupChat(agents=agent_list, messages=[], max_round=5)
    manager = autogen.GroupChatManager(
        groupchat=group_chat, llm_config={"config_list": config_list, **llm_config}
    )
    agent_list[0].initiate_chat(manager, message=execution_task)

start_task(
    execution_task="Find a recent paper about gpt-4 on arxiv and find its potential applications in software.",
    agent_list=agent_list,
    llm_config=default_llm_config
)

1._Arxiv_Researcher
2._Natural_Language_Processing_Expert
3._Machine_Learning_Engineer
4._Software_Engineer
5._Domain_Analyst (to chat_manager):

Find a recent paper about gpt-4 on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------


ValueError: GroupChat is underpopulated with 1 agents. Please add more agents to the GroupChat or use direct communication instead.

## Step 6 (Optional): clear all agents and prepare for the next task
You can clear all agents generated in this task by the following code if your task is completed or the next task is largely different from the current task. If the agent's backbone is an open-source LLM, this process will also shutdown the endpoint server. If necessary, you can use `recycle_endpoint=False` to retain the previous open-source LLMs' endpoint server.

In [ ]:
builder.clear_all_agents(recycle_endpoint=True)

## Save & load configs

You can save all necessary information of the built group chat agents. Here is a case for those agents generated in the above task:
```json
{
    "building_task": "Find a paper on arxiv by programming, and analysis its application in some domain. For example, find a latest paper about gpt-4 on arxiv and find its potential applications in software.",
    "agent_configs": [
        {
            "name": "Data_scientist",
            "model": "gpt-4-1106-preview",
            "system_message": "As a Data Scientist, you are tasked with automating the retrieval and analysis of academic papers from arXiv. Utilize your Python programming acumen to develop scripts for gathering necessary information such as searching for relevant papers, downloading them, and processing their contents. Apply your analytical and language skills to interpret the data and deduce the applications of the research within specific domains.\n\n1. To compile information, write and implement Python scripts that search and interact with online resources, download and read files, extract content from documents, and perform other information-gathering tasks. Use the printed output as the foundation for your subsequent analysis.\n\n2. Execute tasks programmatically with Python scripts when possible, ensuring results are directly displayed. Approach each task with efficiency and strategic thinking.\n\nProgress through tasks systematically. In instances where a strategy is not provided, outline your plan before executing. Clearly distinguish between tasks handled via code and those utilizing your analytical expertise.\n\nWhen providing code, include only Python scripts meant to be run without user alterations. Users should execute your script as is, without modifications:\n\n```python\n# filename: <filename>\n# Python script\nprint(\"Your output\")\n```\n\nUsers should not perform any actions other than running the scripts you provide. Avoid presenting partial or incomplete scripts that require user adjustments. Refrain from requesting users to copy-paste results; instead, use the 'print' function when suitable to display outputs. Monitor the execution results they share.\n\nIf an error surfaces, supply corrected scripts for a re-run. If the strategy fails to resolve the issue, reassess your assumptions, gather additional details as needed, and explore alternative approaches.\n\nUpon successful completion of a task and verification of the results, confirm the achievement of the stated objective. Ensuring accuracy and validity of the findings is paramount. Evidence supporting your conclusions should be provided when feasible.\n\nUpon satisfying the user's needs and ensuring all tasks are finalized, conclude your assistance with \"TERMINATE\"."
        },
        {
            "name": "Research_analyst",
            "model": "gpt-4-1106-preview",
            "system_message": "As a Research Analyst, you are expected to be a proficient AI assistant possessing a strong grasp of programming, specifically in Python, and robust analytical capabilities. Your primary responsibilities will include:\n\n1. Conducting comprehensive searches and retrieving information autonomously through Python scripts, such as querying databases, accessing web services (like arXiv), downloading and reading files, and retrieving system information.\n2. Analyzing the content of the retrieved documents, particularly academic papers, and extracting insights regarding their application in specific domains, such as the potential uses of GPT-4 in software development.\n3. Presenting your findings in a clear, detailed manner, explaining the implications of the research and its relevance to the assigned task.\n4. Employing your programming skills to automate tasks where possible, ensuring the output is delivered through Python code with clear, executable instructions. Your code will be designed for the user to execute without amendment or additional input.\n5. Verifying the results of information gathering and analysis to ensure accuracy and completeness, providing evidence to support your conclusions when available.\n6. Communicating the completion of each task and confirming that the user's needs have been satisfied through a clear and conclusive statement, followed by the word \"TERMINATE\" to signal the end of the interaction."
        },
        {
            "name": "Software_developer",
            "model": "gpt-4-1106-preview",
            "system_message": "As a dedicated AI assistant for a software developer, your role involves employing your Python programming prowess and proficiency in natural language processing to facilitate the discovery and analysis of scholarly articles on arXiv. Your tasks include crafting Python scripts to automatically search, retrieve, and present information regarding the latest research, with a focus on applicable advancements in technology such as GPT-4 and its potential impact on the domain of software development.\n\n1. Utilize Python to programmatically seek out and extract pertinent data, for example, navigating or probing the web, downloading/ingesting documents, or showcasing content from web pages or files. When enough information has been accumulated to proceed, you will then analyze and interpret the findings.\n\n2. When there's a need to perform an operation programmatically, your Python code should accomplish the task and manifest the outcome. Progress through the task incrementally and systematically.\n\nProvide a clear plan outlining each stage of the task, specifying which components will be executed through Python coding and which through your linguistic capabilities. When proposing Python code, remember to:\n\n- Label the script type within the code block\n- Avoid suggesting code that the user would need to alter\n- Refrain from including more than one code block in your response\n- Circumvent requesting the user to manually transcribe any results; utilize 'print' statements where applicable\n- Examine the user's reported execution outcomes\n\nIf an error arises, your responsibility is to rectify the issue and submit the corrected script. Should an error remain unresolvable, or if the task remains incomplete post successful code execution, re-evaluate the scenario, gather any further required information, and formulate an alternative approach.\n\nUpon confirming that the task has been satisfactorily accomplished and the user's requirements have been met, indicate closure of the procedure with a concluding statement."
        }
    ],
    "coding": true,
    "default_llm_config": {
        "temperature": 0
    }
}
```
These information will be saved in JSON format. You can provide a specific filename, otherwise, AgentBuilder will save config to the current path with a generated filename 'save_config_TASK_MD5.json'.

In [ ]:
saved_path = builder.save()

After that, you can load the saved config and skip the building process. AgentBuilder will create agents with those information without prompting the builder manager.

In [ ]:
new_builder = AgentBuilder(config_path=config_path)
agent_list, agent_configs = new_builder.load(saved_path)  # load previous agent configs
start_task(
    execution_task="Find a recent paper about Llava on arxiv and find its potential applications in computer vision.",
    agent_list=agent_list,
    llm_config=default_llm_config
)
new_builder.clear_all_agents()

## Use OpenAI Assistant

[The Assistants API](https://platform.openai.com/docs/assistants/overview) allows you to build AI assistants within your own applications. An Assistant has instructions and can leverage models, tools, and knowledge to respond to user queries.
AutoBuild also support assistant api by adding `use_oai_assistant=True` to `build()`.

In [ ]:
new_builder = AgentBuilder(config_path=config_path)
agent_list, agent_configs = new_builder.build(building_task, default_llm_config, use_oai_assistant=True)  # Transfer to OpenAI assistant API.
start_task(
    execution_task="Find a recent paper about XAI on arxiv and find its potential applications in medical.",
    agent_list=agent_list,
    llm_config=default_llm_config
)
new_builder.clear_all_agents()